<h1>Segmenting and Clustering Neighborhoods in Toronto</h1>

***Import Lib***

In [1]:
#!conda install -c conda-forge geocoder --yes 
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

import geocoder # import geocoder
import pandas as pd # library for data analsysis
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


<h2>Load and Prepare Data</h2>

In [2]:
!wget -q -O 'toronto_data.txt' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print('Data downloaded!')

Data downloaded!


***Read File***

In [3]:
with open('toronto_data.txt') as file_data:
    toronto_list_data = file_data.readlines()

***Select only Toronto table Data***

In [4]:
toronto_html_data = ''.join(toronto_list_data)

startIndex = toronto_html_data.find("<table")
endIndex = toronto_html_data.find("</table>")


toronto_html_data = toronto_html_data[startIndex:endIndex].replace("\n","")                 
                                  

***Use BeautifilSoup to parse html data to table***

In [5]:

table_data = [[cell.text for cell in row("td")]
                         for row in BeautifulSoup(toronto_html_data, "lxml")("tr")]


In [6]:
table_data[0:5]

[[],
 ['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village']]

***Create DataFrame***

In [7]:
# define the dataframe columns
column_names = ['PostalCode','Borough', 'Neighborhood', 'PostalCode-Neighborhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

***Fill Data into DataFrame***

In [8]:
for data in table_data:
    
    #check data exist and Borough is assigned
    if data and data[1] != "Not assigned":
        PostalCode = data[0]
        borough = data[1] 
        
        #check Neighborhood is assigned
        if data[2] == "Not assigned":
            neighborhood_name = data[1]
        else:    
            neighborhood_name = data[2]

        neighborhoods = neighborhoods.append({'PostalCode': PostalCode,
                                              'Borough': borough,
                                              'Neighborhood': neighborhood_name,
                                              'PostalCode-Neighborhood': PostalCode+"-"+neighborhood_name}, ignore_index=True)

In [9]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood,PostalCode-Neighborhood
0,M3A,North York,Parkwoods,M3A-Parkwoods
1,M4A,North York,Victoria Village,M4A-Victoria Village
2,M5A,Downtown Toronto,Harbourfront,M5A-Harbourfront
3,M5A,Downtown Toronto,Regent Park,M5A-Regent Park
4,M6A,North York,Lawrence Heights,M6A-Lawrence Heights


In [10]:
neighborhoods.shape

(212, 4)

***Get Geo Location***

In [11]:
def getLocation(postal_code):
    # initialize your variable to None
    count = 0;
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng
      count += 1
      if(count%50 == 0):
          print(count)

    return lat_lng_coords


Because of long run loop.

So I decide to collect location by use "Geospatial_Coordinates.csv" 

In [12]:
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [13]:
geoLocationDF = pd.read_csv("Geospatial_Coordinates.csv")
geoLocationDF.set_index('Postal Code', inplace=True)

In [14]:
def getLocationFromDF(row):
    return geoLocationDF.loc[row['PostalCode']]

***Apply function to get Location***

In [15]:
locationApply = neighborhoods.apply(getLocationFromDF, axis=1)
neighborhoods = neighborhoods.join(locationApply)
neighborhoods.head()

,PostalCode,Borough,Neighborhood,PostalCode-Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,M3A-Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,M4A-Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,M5A-Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,M5A-Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,M6A-Lawrence Heights,43.718518,-79.464763


In [16]:
neighborhoods.shape

(212, 6)

<h2>Explore and cluster the neighborhoods in Toronto.</h2>

Use only Borough contain "Toronto"

In [17]:
neighborhoods = neighborhoods[neighborhoods['Borough'].str.contains("Toronto")]
#neighborhoods = neighborhoods[neighborhoods['PostalCode'].str.contains("M5")] #more fill for less query test
neighborhoods.reset_index(drop=True, inplace=True)
neighborhoods.head()

,PostalCode,Borough,Neighborhood,PostalCode-Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,M5A-Harbourfront,43.654260,-79.360636
1,M5A,Downtown Toronto,Regent Park,M5A-Regent Park,43.654260,-79.360636
2,M5B,Downtown Toronto,Ryerson,M5B-Ryerson,43.657162,-79.378937
3,M5B,Downtown Toronto,Garden District,M5B-Garden District,43.657162,-79.378937
4,M5C,Downtown Toronto,St. James Town,M5C-St. James Town,43.651494,-79.375418


In [18]:
neighborhoods.shape

(74, 6)

<h3>Toronto Map</h3>

In [19]:
address = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [20]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<h3>Four Square</h3>

In [21]:
CLIENT_ID = 'FOKUNLWX5YFP3DJRD5VKM2K0Z0OUMQRDVQY43TDNYEBBELZH' # your Foursquare ID
CLIENT_SECRET = 'ZRWAVUNZI3WRV2XHIXT5O3KOJRZMMUSNU2FJAJKQLTUNM05X' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FOKUNLWX5YFP3DJRD5VKM2K0Z0OUMQRDVQY43TDNYEBBELZH
CLIENT_SECRET:ZRWAVUNZI3WRV2XHIXT5O3KOJRZMMUSNU2FJAJKQLTUNM05X


Because of they have some duplicate data for the same neighborhood but difference postal code so
I combine  postal code and neighborhood togather to make unique.

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    count = 0
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        count +=1
        print(count," ",name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['PostalCode-Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
toronto_venues = getNearbyVenues(names=neighborhoods['PostalCode']+"-"+neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

1   M5A-Harbourfront
2   M5A-Regent Park
3   M5B-Ryerson
4   M5B-Garden District
5   M5C-St. James Town
6   M4E-The Beaches
7   M5E-Berczy Park
8   M5G-Central Bay Street
9   M6G-Christie
10   M5H-Adelaide
11   M5H-King
12   M5H-Richmond
13   M6H-Dovercourt Village
14   M6H-Dufferin
15   M5J-Harbourfront East
16   M5J-Toronto Islands
17   M5J-Union Station
18   M6J-Little Portugal
19   M6J-Trinity
20   M4K-The Danforth West
21   M4K-Riverdale
22   M5K-Design Exchange
23   M5K-Toronto Dominion Centre
24   M6K-Brockton
25   M6K-Exhibition Place
26   M6K-Parkdale Village
27   M4L-The Beaches West
28   M4L-India Bazaar
29   M5L-Commerce Court
30   M5L-Victoria Hotel
31   M4M-Studio District
32   M4N-Lawrence Park
33   M5N-Roselawn
34   M4P-Davisville North
35   M5P-Forest Hill North
36   M5P-Forest Hill West
37   M6P-High Park
38   M6P-The Junction South
39   M4R-North Toronto West
40   M5R-The Annex
41   M5R-North Midtown
42   M5R-Yorkville
43   M6R-Parkdale
44   M6R-Roncesvalles
45   M4S

In [24]:
print(toronto_venues.shape)
toronto_venues.head()

(3303, 7)


,PostalCode-Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M5A-Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A-Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A-Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
3,M5A-Harbourfront,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,M5A-Harbourfront,43.65426,-79.360636,Cooper Koo YMCA,43.653191,-79.357947,Gym / Fitness Center


In [25]:
toronto_venues.groupby('PostalCode-Neighborhood').count().shape

(74, 6)

In [26]:
toronto_venues.groupby('PostalCode-Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
PostalCode-Neighborhood,,,,,,
M4E-The Beaches,3,3,3,3,3,3
M4K-Riverdale,42,42,42,42,42,42
M4K-The Danforth West,42,42,42,42,42,42
M4L-India Bazaar,20,20,20,20,20,20
M4L-The Beaches West,20,20,20,20,20,20
M4M-Studio District,41,41,41,41,41,41
M4N-Lawrence Park,4,4,4,4,4,4
M4P-Davisville North,8,8,8,8,8,8
M4R-North Toronto West,20,20,20,20,20,20


Analyze Neighborhood

In [27]:
toronto_venues['Venue Category'].unique

<bound method Series.unique of 0                                Bakery
1                           Coffee Shop
2                                   Spa
3                        Breakfast Spot
4                  Gym / Fitness Center
5                            Restaurant
6                        Breakfast Spot
7                                   Pub
8                                  Park
9                         Historic Site
10                       Chocolate Shop
11                       Farmers Market
12                   Mexican Restaurant
13                          Coffee Shop
14                          Coffee Shop
15                         Dessert Shop
16                               Bakery
17                          Coffee Shop
18                   Mexican Restaurant
19                Performing Arts Venue
20                                 Park
21                     Greek Restaurant
22                    French Restaurant
23                          Coffee Shop
24       

In [28]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot.head()

,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [29]:
# add neighborhood column back to dataframe
toronto_onehot['PostalCode-Neighborhood'] = toronto_venues['PostalCode-Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,PostalCode-Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M5A-Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,M5A-Harbourfront,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

group row by PostalCode-Neighborhood

In [30]:
toronto_grouped = toronto_onehot.groupby('PostalCode-Neighborhood').mean().reset_index()
#toronto_grouped

In [31]:
toronto_grouped.shape

(74, 235)

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['PostalCode-Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode-Neighborhood'] = toronto_grouped['PostalCode-Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,PostalCode-Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E-The Beaches,Coffee Shop,Pub,Neighborhood,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
1,M4K-Riverdale,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Yoga Studio,Caribbean Restaurant,Dessert Shop,Diner,Bakery
2,M4K-The Danforth West,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Yoga Studio,Caribbean Restaurant,Dessert Shop,Diner,Bakery
3,M4L-India Bazaar,Park,Pet Store,Steakhouse,Burrito Place,Sushi Restaurant,Burger Joint,Pub,Ice Cream Shop,Fish & Chips Shop,Hotel
4,M4L-The Beaches West,Park,Pet Store,Steakhouse,Burrito Place,Sushi Restaurant,Burger Joint,Pub,Ice Cream Shop,Fish & Chips Shop,Hotel


<h3>Cluster by K-Mean</h3>

In [34]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('PostalCode-Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 0, 0, 1, 0, 0, 1, 1], dtype=int32)

In [35]:
toronto_merged = neighborhoods

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('PostalCode-Neighborhood'), on='PostalCode-Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,PostalCode-Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Harbourfront,M5A-Harbourfront,43.654260,-79.360636,1,Coffee Shop,Bakery,Park,Café,Restaurant,Pub,Mexican Restaurant,Breakfast Spot,Theater,Health Food Store
1,M5A,Downtown Toronto,Regent Park,M5A-Regent Park,43.654260,-79.360636,1,Coffee Shop,Bakery,Park,Café,Restaurant,Pub,Mexican Restaurant,Breakfast Spot,Theater,Health Food Store
2,M5B,Downtown Toronto,Ryerson,M5B-Ryerson,43.657162,-79.378937,1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Bar,Japanese Restaurant,Middle Eastern Restaurant,Restaurant,Movie Theater,Italian Restaurant
3,M5B,Downtown Toronto,Garden District,M5B-Garden District,43.657162,-79.378937,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Bar,Japanese Restaurant,Middle Eastern Restaurant,Restaurant,Movie Theater,Italian Restaurant
4,M5C,Downtown Toronto,St. James Town,M5C-St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Restaurant,Clothing Store,Hotel,Gastropub,Bakery,Cosmetics Shop,Italian Restaurant,Cocktail Bar


<h3>Map Cluster</h3>

In [36]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h3>Examine Clusters</h3>

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Garden District,0,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Bar,Japanese Restaurant,Middle Eastern Restaurant,Restaurant,Movie Theater,Italian Restaurant
4,St. James Town,0,Coffee Shop,Café,Restaurant,Clothing Store,Hotel,Gastropub,Bakery,Cosmetics Shop,Italian Restaurant,Cocktail Bar
6,Berczy Park,0,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Beer Bar,Cheese Shop,Steakhouse,Restaurant,Café,Farmers Market
7,Central Bay Street,0,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Bar,Bubble Tea Shop,Burger Joint,Japanese Restaurant,Ice Cream Shop,Falafel Restaurant
39,The Annex,0,Coffee Shop,Café,Sandwich Place,Pizza Place,French Restaurant,Cosmetics Shop,Pub,Burger Joint,Metro Station,Liquor Store
40,North Midtown,0,Coffee Shop,Café,Sandwich Place,Pizza Place,French Restaurant,Cosmetics Shop,Pub,Burger Joint,Metro Station,Liquor Store


In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Harbourfront,1,Coffee Shop,Bakery,Park,Café,Restaurant,Pub,Mexican Restaurant,Breakfast Spot,Theater,Health Food Store
1,Regent Park,1,Coffee Shop,Bakery,Park,Café,Restaurant,Pub,Mexican Restaurant,Breakfast Spot,Theater,Health Food Store
2,Ryerson,1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Bar,Japanese Restaurant,Middle Eastern Restaurant,Restaurant,Movie Theater,Italian Restaurant
5,The Beaches,1,Coffee Shop,Pub,Neighborhood,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
8,Christie,1,Grocery Store,Café,Park,Nightclub,Diner,Baby Store,Italian Restaurant,Athletics & Sports,Restaurant,Coffee Shop
9,Adelaide,1,Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant,Restaurant,Gym,Cosmetics Shop,Hotel,Bar
12,Dovercourt Village,1,Bakery,Supermarket,Gym / Fitness Center,Athletics & Sports,Gas Station,Fast Food Restaurant,Liquor Store,Discount Store,Middle Eastern Restaurant,Music Venue
13,Dufferin,1,Bakery,Supermarket,Gym / Fitness Center,Athletics & Sports,Gas Station,Fast Food Restaurant,Liquor Store,Discount Store,Middle Eastern Restaurant,Music Venue
14,Harbourfront East,1,Coffee Shop,Hotel,Aquarium,Pizza Place,Café,Scenic Lookout,Sports Bar,Brewery,Italian Restaurant,Fried Chicken Joint
15,Toronto Islands,1,Coffee Shop,Hotel,Aquarium,Pizza Place,Café,Scenic Lookout,Sports Bar,Brewery,Italian Restaurant,Fried Chicken Joint


In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,Moore Park,2,Restaurant,Park,Playground,Tennis Court,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Yoga Studio
50,Summerhill East,2,Restaurant,Park,Playground,Tennis Court,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Yoga Studio
51,Chinatown,2,Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Bar,Vietnamese Restaurant,Bakery,Mexican Restaurant,Coffee Shop,Dumpling Restaurant,Ice Cream Shop
52,Grange Park,2,Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Bar,Vietnamese Restaurant,Bakery,Mexican Restaurant,Coffee Shop,Dumpling Restaurant,Ice Cream Shop
53,Kensington Market,2,Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Bar,Vietnamese Restaurant,Bakery,Mexican Restaurant,Coffee Shop,Dumpling Restaurant,Ice Cream Shop
54,Deer Park,2,Coffee Shop,Convenience Store,Pub,Sushi Restaurant,Bagel Shop,Fried Chicken Joint,Sports Bar,American Restaurant,Pizza Place,Vietnamese Restaurant
55,Forest Hill SE,2,Coffee Shop,Convenience Store,Pub,Sushi Restaurant,Bagel Shop,Fried Chicken Joint,Sports Bar,American Restaurant,Pizza Place,Vietnamese Restaurant


In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,King,3,Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant,Restaurant,Gym,Cosmetics Shop,Hotel,Bar
11,Richmond,3,Coffee Shop,Café,American Restaurant,Steakhouse,Thai Restaurant,Restaurant,Gym,Cosmetics Shop,Hotel,Bar
17,Little Portugal,3,Bar,Café,Restaurant,Coffee Shop,Bakery,Pizza Place,Cocktail Bar,French Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant


In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(6, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
38,North Toronto West,4,Clothing Store,Sporting Goods Shop,Coffee Shop,Yoga Studio,Chinese Restaurant,Dessert Shop,Diner,Fast Food Restaurant,Gift Shop,Mexican Restaurant
